##### Import Libs

In [1]:
import pandas as pd
import swifter
from transformers import pipeline
from datasets import load_dataset

c:\Users\djbac\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load the Dataset

In [2]:
df = pd.read_csv("datasets\mtsamples.csv", index_col=0)

In [3]:
# Filtering to 12 classes for simplicity
list_med_sp = [
    " Allergy / Immunology",
    " Bariatrics",
    " Cardiovascular / Pulmonary",
    " Urology",
    " Dentistry",
    " Rheumatology",
    " Radiology",
    " Psychiatry / Psychology",
    " Podiatry",
    " Orthopedic",
    " Opthalmology",
    " Neurology"
]
# Filter the DataFrame
df = df[df['medical_specialty'].isin(list_med_sp)]

# df_stratified, _ = train_test_split(df, train_size=0.03, stratify=df['medical_specialty'])

#### Specify the candidate labels for the ZSC

In [4]:
candidate_labels = df["medical_specialty"].unique().tolist()

#### Create the Classifier

In [5]:
classifier = (
    pipeline(task="zero-shot-classification",
             model="tasksource/deberta-small-long-nli"
    )
)

In [6]:
# Function to classify a single description
def classify_description(description):
    result = classifier(description, candidate_labels)
    # Find the index of the maximum score
    max_index = result['scores'].index(max(result['scores']))
    # Return the label with the highest score
    return result['labels'][max_index], result['scores'][max_index]

#### Zero-shot

In [172]:
df[['zero_shot_class', 'score']] = (
    df['description']
    .swifter.progress_bar(enable=True)
    .apply(classify_description)  # Apply function directly
    .apply(pd.Series)  # Convert tuple/list output into DataFrame
)

Pandas Apply: 100%|██████████| 1543/1543 [49:52<00:00,  1.94s/it]


In [174]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Assuming 'y_true' is the actual label and 'y_pred' is the predicted label
y_true = df['medical_specialty']
y_pred = df['zero_shot_class']

# Get accuracy score
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Get classification report
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Get confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Accuracy: 0.53
Classification Report:
                             precision    recall  f1-score   support

       Allergy / Immunology       0.16      1.00      0.27         7
                 Bariatrics       0.75      0.83      0.79        18
 Cardiovascular / Pulmonary       0.81      0.23      0.36       372
                  Dentistry       0.76      0.81      0.79        27
                  Neurology       0.71      0.31      0.44       223
                 Orthopedic       0.71      0.74      0.72       355
                   Podiatry       0.55      0.38      0.45        47
    Psychiatry / Psychology       0.78      0.74      0.76        53
                  Radiology       0.31      0.76      0.44       273
               Rheumatology       0.18      0.40      0.25        10
                    Urology       0.93      0.59      0.73       158

                   accuracy                           0.53      1543
                  macro avg       0.60      0.62      0.54     